In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-6.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...  dam  moa  \
0    prop-6      452    2    2    0    5    9     1   1   4  ...  0.0    0   
1    prop-6      452    3    1    0    1    3     3   1   0  ...  0.0    0   
2    prop-6      452   13    2    0    8   17    78   0   8  ...  0.0    0   
3    prop-6      452    2    2    0    5    7     1   1   4  ...  0.0    0   
4    prop-6      452   23    1    0   10   23   253   2   8  ...  0.0    0   
..      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...  ...  ...   
655  prop-6      454    9    2    0   10   20    36   5  10  ...  0.0    0   
656  prop-6      454    6    2    0    7   15    15   2   7  ...  0.0    0   
657  prop-6      454    6    1    0    5   29    11   2   4  ...  0.0    0   
658  prop-6      454    6    1    0    3   31     1   1   2  ...  1.0    1   
659  prop-6      454   29    1    0   11   50   180  10   1  ...  1.0    0   

          mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0    0.909091  0.750000   1    1  23.500000       1  0.5000    1  
1    0.000000  1.000000   0    0   0.000000       1  1.0000    0  
2    0.586207  1.000000   1    3   2.769231       1  0.9231    0  
3    0.909091  0.750000   1    2   7.500000       1  0.5000    0  
4    0.000000  0.255435   0    0   0.000000       1  1.0000    0  
..        ...       ...  ..  ...        ...     ...     ...  ...  
655  0.200000  0.377778   0    0   7.888889       1  0.8889    0  
656  0.285714  0.400000   0    0   6.666667       1  0.8333    0  
657  0.000000  0.400000   0    0  26.666667       4  1.5000    0  
658  0.000000  0.333333   0    0  32.500000       3  1.3333    0  
659  0.000000  0.226601   1    1  13.206897       7  1.2759    0  

[660 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 452]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...  dam  moa  \
0    prop-6      452    2    2    0    5    9     1   1   4  ...  0.0    0   
1    prop-6      452    3    1    0    1    3     3   1   0  ...  0.0    0   
2    prop-6      452   13    2    0    8   17    78   0   8  ...  0.0    0   
3    prop-6      452    2    2    0    5    7     1   1   4  ...  0.0    0   
4    prop-6      452   23    1    0   10   23   253   2   8  ...  0.0    0   
..      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...  ...  ...   
251  prop-6      452    7    1    0    5    7    21   3   2  ...  0.0    0   
252  prop-6      452    2    1    0    3    2     1   2   1  ...  0.0    0   
253  prop-6      452    4    1    0    3    8     0   2   1  ...  1.0    0   
254  prop-6      452    6    1    0    6    6    15   4   2  ...  0.0    0   
255  prop-6      452    2    2    0    4    5     1   0   4  ...  0.0    0   

          mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0    0.909091  0.750000   1    1  23.500000       1  0.5000    1  
1    0.000000  1.000000   0    0   0.000000       1  1.0000    0  
2    0.586207  1.000000   1    3   2.769231       1  0.9231    0  
3    0.909091  0.750000   1    2   7.500000       1  0.5000    0  
4    0.000000  0.255435   0    0   0.000000       1  1.0000    0  
..        ...       ...  ..  ...        ...     ...     ...  ...  
251  0.000000  0.535714   0    0   0.000000       1  1.0000    0  
252  0.000000  0.750000   0    0   0.000000       1  1.0000    0  
253  0.000000  0.500000   0    0   6.000000       1  0.7500    0  
254  0.000000  0.333333   0    0   0.000000       1  1.0000    0  
255  0.666667  0.750000   0    0   4.500000       1  0.5000    0  

[256 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  moa  \
204    4    1    0    6    4     6   5   1    4  2.000000    4  0.000000    0   
125    8    2    0   20   52    18   1  19    2  0.785714  360  1.000000    0   
87     4    1    0    4    5     0   4   0    3  0.833333   22  1.000000    0   
150    4    1    0   15    4     6  14   1    4  2.000000    4  0.000000    0   
142   19    1    0   31  135   115   0  31   16  0.833333  798  1.000000    8   
..   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...  ...   
205    2    1    0    5    2     1   4   1    2  2.000000    2  0.000000    0   
43     5    4    2   11    8     8  11   0    4  1.000000   26  1.000000    0   
185   15    1    0   13   42    41   0  13   15  0.846939  221  0.857143    6   
27     6    2    0   11   43     1   0  11    2  1.000000  243  1.000000    1   
129    1    1    0    3    1     0   2   1    1  2.000000    1  0.000000    0   

          mfa       cam  ic  cbm        amc  max_cc  avg_cc  
204  0.000000  0.687500   0    0   0.000000       1  1.0000  
125  0.730769  0.325000   1    3  43.500000       6  2.5000  
87   0.000000  0.750000   0    0   4.000000       1  0.5000  
150  0.000000  0.625000   0    0   0.000000       1  1.0000  
142  0.000000  0.136842   0    0  40.526316      13  2.3158  
..        ...       ...  ..  ...        ...     ...     ...  
205  0.000000  0.666667   0    0   0.000000       1  1.0000  
43   0.894737  0.600000   0    0   3.800000       1  0.4000  
185  0.000000  0.200000   0    0  13.266667       5  1.3333  
27   0.750000  0.440000   2    2  37.833333       4  1.8333  
129  0.000000  1.000000   0    0   0.000000       1  1.0000  

[204 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  moa  \
139    2    2    0    5    7     1   1   4    1  2.000000   17  0.000000    0   
141    1    1    0    0    2     0   0   0    0  2.000000   13  0.000000    0   
133   10    1    0    8   21     0   8   0   10  0.666667  140  1.000000    0   
83     2    2    0    8   12     1   0   8    2  1.000000   59  1.000000    0   
162    6    1    0   20   47    13   0  20    4  1.160000  274  1.000000    1   
190    5    1    0   39   18     0  36   3    3  0.250000   75  1.000000    1   
242   13    2    0    9   19    78   0   9   13  2.000000   81  0.000000    0   
50    13    2    0    9   19    78   0   9   13  2.000000   57  0.000000    0   
207    6    2    0    2   14    11   1   1    4  0.800000  105  0.400000    4   
2     13    2    0    8   17    78   0   8   13  2.000000   49  0.000000    0   
203    3    1    0    3    3     3   1   2    3  2.000000    3  0.000000    0   
199    2    2    0    7   11     1   0   7    2  1.000000   35  1.000000    0   
26    16    1    0   15   50    68   0  15   12  0.816667  365  1.000000    7   
100   13    2    0    9   19    78   0   9   13  2.000000   81  0.000000    0   
188    2    2    0    7   11     1   0   7    2  1.000000   46  1.000000    0   
248   23    1    0    8   23   253   1   7   23  2.000000   23  0.000000    0   
101   12    1    0   17   50    18  12   7    7  0.727273  253  1.000000    0   
97     9    1    0    7   16     6   7   0    8  0.708333  104  1.000000    0   
32    14    2    0    7   17    91   0   7   14  2.000000   54  0.000000    0   
135    8    1    0    9    9    14   8   1    7  0.857143   46  1.000000    1   
164    4    1    0    0    5     0   0   0    4  1.000000   23  0.333333    0   
14    20    1    0   23   94   126   0  23   11  0.885965  771  1.000000    5   
138   13    2    0    8   17    78   0   8   13  2.000000   49  0.000000    0   
1      3    1    0    1    3     3   1   0    3  2.000000    3  0.000000    0   
96     1    1    0    0    2     0   0   0    1  2.000000   32  0.000000    0   
136   17    1    0   76   25    88  74   2   15  0.854167  163  1.000000    2   
69    11    1    0   18   49    51   1  17    4  1.076923  350  1.000000    1   
179    6    2    0   24   55     0   2  23    2  0.640000  291  1.000000    1   
37     2    2    0    5    7     1   1   4    1  2.000000   17  0.000000    0   
222    3    1    0    3    3     3   2   1    3  2.000000    3  0.000000    0   
206   22    1    0   25   86   167   0  25   19  0.869048  464  1.000000    7   
197   20    1    0    6   42   118   4   3   20  0.810526  264  1.000000    1   
70    37    1    0   40   60   492  38   6   25  0.840278  441  1.000000    2   
19     2    1    0    3    7     1   2   1    1  2.000000   32  0.000000    0   
84     3    1    0    5   17     0   0   5    3  0.500000   94  1.000000    0   
165   15    1    0    7   19    83   6   1   15  0.904762  129  1.000000    1   
92     9    2    0   10   20    36   4  10    9  2.000000   77  0.000000    0   
117    3    5    0    9    6     3   8   1    2  1.500000   16  1.000000    0   
10     2    2    0    7    6     1   6   1    2  2.000000   16  0.000000    0   
253    4    1    0    3    8     0   2   1    4  0.500000   30  1.000000    0   
233   29    1    0   11   50   180  10   1   26  0.726190  418  1.000000    0   
217   20    1    0   19   52   126   0  19   20  0.888158  264  1.000000    8   
178    2    2    0    7   12     1   0   7    2  1.000000  110  1.000000    0   
243    9    2    0   10   20    36   5  10    9  2.000000   80  0.000000    0   
155    6    2    0    9   15    15   3   9    6  2.000000   60  0.000000    0   
137    7    1    0    5    7    21   3   2    7  2.000000    7  0.000000    0   
115    2    3    0    3   19     1   0   3    2  2.000000   91  1.000000    0   
122    2    1    0    6    2     1   4   2    2  2.000000    2  0.000000    0   
80     4    2    0    7   25     2  

In [9]:
y_train = pd.DataFrame(train['bug'])
y_test = pd.DataFrame(test['bug'])

In [10]:
y_train

bug
204    0
125    0
87     0
150    0
142    1
..   ...
205    0
43     0
185    0
27     0
129    0

[204 rows x 1 columns]

In [11]:
y_test

bug
139    0
141    1
133    0
83     0
162    0
190    0
242    0
50     0
207    0
2      0
203    0
199    0
26     0
100    1
188    0
248    0
101    0
97     0
32     0
135    0
164    0
14     1
138    0
1      0
96     0
136    0
69     2
179    1
37     0
222    1
206    1
197    0
70     1
19     0
84     0
165    0
92     0
117    0
10     0
253    0
233    0
217    0
178    0
243    0
155    0
137    0
115    0
122    0
80     0
149    0
55     0
11     0

In [12]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,20
# Preparing the data for the model.
X_train_matrix = x_train.values
X_test_matrix = x_test.values
Y_train_matrix = y_train.values
Y_test_matrix = y_test.values
# Preparing the data for the model.
Ytrainseries = y_train['bug']
Ytestseries = y_test['bug']

In [13]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [14]:
def define_model():
    # Designing and initializing the model.
    #Building the model
    model = Sequential()
    #add model layers
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    #compile model using mse as the loss function
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])
    return model

In [15]:
y = y_test['bug'].tolist()
y

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [16]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/cnn_propv452/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(X_test1, Y_test_matrix)
    total_scores.append(score) 
    predictions = model.predict(X_test1)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
3/3 [==============================] - 0s 10ms/step - loss: 2989.6287 - mse: 2989.6287 - mae: 39.4960 - root_mean_squared_error: 54.6775 - mean_squared_logarithmic_error: 10.9001
Epoch 2/100
3/3 [==============================] - 0s 2ms/step - loss: 2863.4087 - mse: 2863.4087 - mae: 38.5995 - root_mean_squared_error: 53.5108 - mean_squared_logarithmic_error: 10.7451
Epoch 3/100
3/3 [==============================] - 0s 1ms/step - loss: 2738.8494 - mse: 2738.8494 - mae: 37.7103 - root_mean_squared_error: 52.3340 - mean_squared_logarithmic_error: 10.5891
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 2621.4644 - mse: 2621.4644 - mae: 36.8273 - root_mean_squared_error: 51.2002 - mean_squared_logarithmic_error: 10.4307
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: 2508.4148 - mse: 2508.4148 - mae: 35.9562 - root_mean_squared_error: 50.0841 - mean_squared_logarithmic_error: 10.2693
Epoch 6/100
3/3 [======================

3/3 [==============================] - 0s 2ms/step - loss: 60.3462 - mse: 60.3462 - mae: 4.3232 - root_mean_squared_error: 7.7683 - mean_squared_logarithmic_error: 2.0932
Epoch 87/100
3/3 [==============================] - 0s 2ms/step - loss: 57.5276 - mse: 57.5276 - mae: 4.2026 - root_mean_squared_error: 7.5847 - mean_squared_logarithmic_error: 2.0436
Epoch 88/100
3/3 [==============================] - 0s 964us/step - loss: 55.2573 - mse: 55.2573 - mae: 4.0959 - root_mean_squared_error: 7.4335 - mean_squared_logarithmic_error: 1.9966
Epoch 89/100
3/3 [==============================] - 0s 1ms/step - loss: 52.9358 - mse: 52.9358 - mae: 3.9930 - root_mean_squared_error: 7.2757 - mean_squared_logarithmic_error: 1.9526
Epoch 90/100
3/3 [==============================] - 0s 973us/step - loss: 50.7739 - mse: 50.7739 - mae: 3.8919 - root_mean_squared_error: 7.1256 - mean_squared_logarithmic_error: 1.9073
Epoch 91/100
3/3 [==============================] - 0s 920us/step - loss: 48.7652 - mse: 

3/3 [==============================] - 0s 2ms/step - loss: 0.1373 - mse: 0.1373 - mae: 0.1275 - root_mean_squared_error: 0.3705 - mean_squared_logarithmic_error: 0.0624
Epoch 74/100
3/3 [==============================] - 0s 993us/step - loss: 0.1373 - mse: 0.1373 - mae: 0.1275 - root_mean_squared_error: 0.3705 - mean_squared_logarithmic_error: 0.0624
Epoch 75/100
3/3 [==============================] - 0s 1ms/step - loss: 0.1373 - mse: 0.1373 - mae: 0.1275 - root_mean_squared_error: 0.3705 - mean_squared_logarithmic_error: 0.0624
Epoch 76/100
3/3 [==============================] - 0s 1ms/step - loss: 0.1373 - mse: 0.1373 - mae: 0.1275 - root_mean_squared_error: 0.3705 - mean_squared_logarithmic_error: 0.0624
Epoch 77/100
3/3 [==============================] - 0s 2ms/step - loss: 0.1373 - mse: 0.1373 - mae: 0.1275 - root_mean_squared_error: 0.3705 - mean_squared_logarithmic_error: 0.0624
Epoch 78/100
3/3 [==============================] - 0s 1ms/step - loss: 0.1373 - mse: 0.1373 - mae: 0

3/3 [==============================] - 0s 1ms/step - loss: 0.1764 - mse: 0.1764 - mae: 0.1518 - root_mean_squared_error: 0.4200 - mean_squared_logarithmic_error: 0.0761
Epoch 61/100
3/3 [==============================] - 0s 2ms/step - loss: 0.1759 - mse: 0.1759 - mae: 0.1516 - root_mean_squared_error: 0.4194 - mean_squared_logarithmic_error: 0.0760
Epoch 62/100
3/3 [==============================] - 0s 1ms/step - loss: 0.1754 - mse: 0.1754 - mae: 0.1515 - root_mean_squared_error: 0.4188 - mean_squared_logarithmic_error: 0.0759
Epoch 63/100
3/3 [==============================] - 0s 2ms/step - loss: 0.1748 - mse: 0.1748 - mae: 0.1512 - root_mean_squared_error: 0.4181 - mean_squared_logarithmic_error: 0.0758
Epoch 64/100
3/3 [==============================] - 0s 1ms/step - loss: 0.1744 - mse: 0.1744 - mae: 0.1511 - root_mean_squared_error: 0.4176 - mean_squared_logarithmic_error: 0.0757
Epoch 65/100
3/3 [==============================] - 0s 2ms/step - loss: 0.1738 - mse: 0.1738 - mae: 0.1

3/3 [==============================] - 0s 1ms/step - loss: 733.1436 - mse: 733.1436 - mae: 17.7752 - root_mean_squared_error: 27.0766 - mean_squared_logarithmic_error: 6.3319
Epoch 45/100
3/3 [==============================] - 0s 1ms/step - loss: 714.8680 - mse: 714.8680 - mae: 17.5317 - root_mean_squared_error: 26.7370 - mean_squared_logarithmic_error: 6.2706
Epoch 46/100
3/3 [==============================] - 0s 1ms/step - loss: 697.0827 - mse: 697.0827 - mae: 17.3013 - root_mean_squared_error: 26.4023 - mean_squared_logarithmic_error: 6.2133
Epoch 47/100
3/3 [==============================] - 0s 2ms/step - loss: 681.9302 - mse: 681.9302 - mae: 17.0902 - root_mean_squared_error: 26.1138 - mean_squared_logarithmic_error: 6.1589
Epoch 48/100
3/3 [==============================] - 0s 1ms/step - loss: 664.4167 - mse: 664.4167 - mae: 16.8566 - root_mean_squared_error: 25.7763 - mean_squared_logarithmic_error: 6.0995
Epoch 49/100
3/3 [==============================] - 0s 2ms/step - loss: 6

...

2/2 [==============================] - 0s 2ms/step - loss: 180.4722 - mse: 180.4722 - mae: 5.1063 - root_mean_squared_error: 13.4340 - mean_squared_logarithmic_error: 1.9450
score of 25th run is: 180.47222900390625, 180.47219848632812, 5.106348037719727, 13.43399429321289, 1.944958209991455
Run no: 26
Epoch 1/100
3/3 [==============================] - 0s 24ms/step - loss: 139.8086 - mse: 139.8086 - mae: 3.7723 - root_mean_squared_error: 11.8241 - mean_squared_logarithmic_error: 1.4693
Epoch 2/100
3/3 [==============================] - 0s 1ms/step - loss: 134.9236 - mse: 134.9236 - mae: 3.6846 - root_mean_squared_error: 11.6157 - mean_squared_logarithmic_error: 1.4438
Epoch 3/100
3/3 [==============================] - 0s 1ms/step - loss: 130.4761 - mse: 130.4761 - mae: 3.6047 - root_mean_squared_error: 11.4226 - mean_squared_logarithmic_error: 1.4193
Epoch 4/100
3/3 [==============================] - 0s 1ms/step - loss: 126.7982 - mse: 126.7982 - mae: 3.5327 - root_mean_squared_error: 1

3/3 [==============================] - 0s 1ms/step - loss: 33.2425 - mse: 33.2425 - mae: 1.4187 - root_mean_squared_error: 5.7656 - mean_squared_logarithmic_error: 0.6461
Epoch 86/100
3/3 [==============================] - 0s 1ms/step - loss: 32.7431 - mse: 32.7431 - mae: 1.4015 - root_mean_squared_error: 5.7222 - mean_squared_logarithmic_error: 0.6379
Epoch 87/100
3/3 [==============================] - 0s 1ms/step - loss: 32.2982 - mse: 32.2982 - mae: 1.3862 - root_mean_squared_error: 5.6831 - mean_squared_logarithmic_error: 0.6306
Epoch 88/100
3/3 [==============================] - 0s 1ms/step - loss: 31.8540 - mse: 31.8540 - mae: 1.3718 - root_mean_squared_error: 5.6439 - mean_squared_logarithmic_error: 0.6237
Epoch 89/100
3/3 [==============================] - 0s 1ms/step - loss: 31.5154 - mse: 31.5154 - mae: 1.3595 - root_mean_squared_error: 5.6139 - mean_squared_logarithmic_error: 0.6178
Epoch 90/100
3/3 [==============================] - 0s 2ms/step - loss: 31.1910 - mse: 31.191

3/3 [==============================] - 0s 1ms/step - loss: 0.2451 - mse: 0.2451 - mae: 0.1673 - root_mean_squared_error: 0.4951 - mean_squared_logarithmic_error: 0.0786
Epoch 73/100
3/3 [==============================] - 0s 1ms/step - loss: 0.2439 - mse: 0.2439 - mae: 0.1668 - root_mean_squared_error: 0.4939 - mean_squared_logarithmic_error: 0.0786
Epoch 74/100
3/3 [==============================] - 0s 1ms/step - loss: 0.2427 - mse: 0.2427 - mae: 0.1664 - root_mean_squared_error: 0.4926 - mean_squared_logarithmic_error: 0.0786
Epoch 75/100
3/3 [==============================] - 0s 2ms/step - loss: 0.2412 - mse: 0.2412 - mae: 0.1660 - root_mean_squared_error: 0.4912 - mean_squared_logarithmic_error: 0.0785
Epoch 76/100
3/3 [==============================] - 0s 1ms/step - loss: 0.2397 - mse: 0.2397 - mae: 0.1656 - root_mean_squared_error: 0.4895 - mean_squared_logarithmic_error: 0.0785
Epoch 77/100
3/3 [==============================] - 0s 1ms/step - loss: 0.2388 - mse: 0.2388 - mae: 0.1

3/3 [==============================] - 0s 3ms/step - loss: 0.1606 - mse: 0.1606 - mae: 0.1463 - root_mean_squared_error: 0.4007 - mean_squared_logarithmic_error: 0.0722
Epoch 60/100
3/3 [==============================] - 0s 3ms/step - loss: 0.1598 - mse: 0.1598 - mae: 0.1460 - root_mean_squared_error: 0.3998 - mean_squared_logarithmic_error: 0.0720
Epoch 61/100
3/3 [==============================] - 0s 1ms/step - loss: 0.1593 - mse: 0.1593 - mae: 0.1458 - root_mean_squared_error: 0.3992 - mean_squared_logarithmic_error: 0.0718
Epoch 62/100
3/3 [==============================] - 0s 2ms/step - loss: 0.1587 - mse: 0.1587 - mae: 0.1455 - root_mean_squared_error: 0.3984 - mean_squared_logarithmic_error: 0.0717
Epoch 63/100
3/3 [==============================] - 0s 3ms/step - loss: 0.1583 - mse: 0.1583 - mae: 0.1454 - root_mean_squared_error: 0.3979 - mean_squared_logarithmic_error: 0.0715
Epoch 64/100
3/3 [==============================] - 0s 2ms/step - loss: 0.1577 - mse: 0.1577 - mae: 0.1

3/3 [==============================] - 0s 2ms/step - loss: 141.9055 - mse: 141.9055 - mae: 3.0431 - root_mean_squared_error: 11.9124 - mean_squared_logarithmic_error: 1.0891
Epoch 45/100
3/3 [==============================] - 0s 1ms/step - loss: 140.1864 - mse: 140.1864 - mae: 3.0188 - root_mean_squared_error: 11.8400 - mean_squared_logarithmic_error: 1.0828
Epoch 46/100
3/3 [==============================] - 0s 1ms/step - loss: 138.2497 - mse: 138.2497 - mae: 2.9944 - root_mean_squared_error: 11.7580 - mean_squared_logarithmic_error: 1.0772
Epoch 47/100
3/3 [==============================] - 0s 1ms/step - loss: 136.5298 - mse: 136.5298 - mae: 2.9685 - root_mean_squared_error: 11.6846 - mean_squared_logarithmic_error: 1.0701
Epoch 48/100
3/3 [==============================] - 0s 1ms/step - loss: 134.8017 - mse: 134.8017 - mae: 2.9449 - root_mean_squared_error: 11.6104 - mean_squared_logarithmic_error: 1.0638
Epoch 49/100
3/3 [==============================] - 0s 1ms/step - loss: 133.32

In [17]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/cnn_propv452/'

In [18]:
total_training_time

[1.4035112857818604,
 1.1741366386413574,
 1.200692892074585,
 1.1800520420074463,
 1.1683154106140137,
 1.1850450038909912,
 1.2024178504943848,
 1.2926692962646484,
 1.1831514835357666,
 1.2565393447875977,
 1.2374951839447021,
 1.329854965209961,
 1.2716784477233887,
 1.2698194980621338,
 1.427525281906128,
 1.2271728515625,
 1.2293312549591064,
 1.0870628356933594,
 1.1104135513305664,
 1.0534260272979736,
 1.1182887554168701,
 1.2551405429840088,
 1.184924602508545,
 1.3221032619476318,
 1.1599020957946777,
 1.2265353202819824,
 1.2693326473236084,
 1.5135033130645752,
 1.5475842952728271,
 1.4239959716796875]

In [19]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 1.250387398401896


In [20]:
total_training_loss_history

[{'loss': [2989.628662109375,
   2863.40869140625,
   2738.849365234375,
   2621.46435546875,
   2508.414794921875,
   2398.312744140625,
   2302.10107421875,
   2210.048095703125,
   2119.10400390625,
   2034.2261962890625,
   1950.2698974609375,
   1873.193359375,
   1795.853759765625,
   1721.6170654296875,
   1652.7308349609375,
   1581.0999755859375,
   1516.1705322265625,
   1452.8358154296875,
   1395.379150390625,
   1341.998291015625,
   1289.7486572265625,
   1239.72119140625,
   1188.9046630859375,
   1131.334716796875,
   1072.7320556640625,
   1021.8440551757812,
   978.5056762695312,
   937.1367797851562,
   897.5961303710938,
   857.0779418945312,
   823.6177368164062,
   791.93994140625,
   761.5953369140625,
   732.63671875,
   703.877685546875,
   676.84423828125,
   648.31787109375,
   625.5642700195312,
   602.7498168945312,
   578.1310424804688,
   554.6034545898438,
   532.9749755859375,
   511.4732666015625,
   489.32086181640625,
   468.4059143066406,
   450.705

In [21]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 196.86147289276124
avg mae of training: 3.7816387449701625


In [22]:
total_scores

[[21.318727493286133,
  21.318727493286133,
  2.059077262878418,
  4.617220878601074,
  1.0217067003250122],
 [0.21153846383094788,
  0.21153846383094788,
  0.17307692766189575,
  0.4599331021308899,
  0.08788689970970154],
 [0.23997043073177338,
  0.23997043073177338,
  0.19646000862121582,
  0.48986777663230896,
  0.10006165504455566],
 [196.42262268066406,
  196.42262268066406,
  7.241370677947998,
  14.015085220336914,
  2.8061394691467285],
 [0.21153846383094788,
  0.21153846383094788,
  0.17307692766189575,
  0.4599331021308899,
  0.08788689970970154],
 [0.21153846383094788,
  0.21153846383094788,
  0.17307692766189575,
  0.4599331021308899,
  0.08788689970970154],
 [0.14988380670547485,
  0.14988380670547485,
  0.2244052141904831,
  0.38714829087257385,
  0.06015921011567116],
 [0.21153846383094788,
  0.21153846383094788,
  0.17307692766189575,
  0.4599331021308899,
  0.08788689970970154],
 [0.21153846383094788,
  0.21153846383094788,
  0.17307692766189575,
  0.4599331021308899,

In [1]:
a = [[21.318727493286133,
  21.318727493286133,
  2.059077262878418,
  4.617220878601074,
  1.0217067003250122],
 [0.21153846383094788,
  0.21153846383094788,
  0.17307692766189575,
  0.4599331021308899,
  0.08788689970970154],
 [0.23997043073177338,
  0.23997043073177338,
  0.19646000862121582,
  0.48986777663230896,
  0.10006165504455566],
 [196.42262268066406,
  196.42262268066406,
  7.241370677947998,
  14.015085220336914,
  2.8061394691467285],
 [0.21153846383094788,
  0.21153846383094788,
  0.17307692766189575,
  0.4599331021308899,
  0.08788689970970154],
 [0.21153846383094788,
  0.21153846383094788,
  0.17307692766189575,
  0.4599331021308899,
  0.08788689970970154],
 [0.14988380670547485,
  0.14988380670547485,
  0.2244052141904831,
  0.38714829087257385,
  0.06015921011567116],
 [0.21153846383094788,
  0.21153846383094788,
  0.17307692766189575,
  0.4599331021308899,
  0.08788689970970154],
 [0.21153846383094788,
  0.21153846383094788,
  0.17307692766189575,
  0.4599331021308899,
  0.08788689970970154],
 [5.42832612991333,
  5.42832612991333,
  0.6093320250511169,
  2.3298768997192383,
  0.31539788842201233],
 [0.1454777717590332,
  0.1454777717590332,
  0.1665557473897934,
  0.3814154863357544,
  0.05820375680923462],
 [21.469449996948242,
  21.469449996948242,
  1.2514338493347168,
  4.633513927459717,
  0.5328952074050903],
 [17.206315994262695,
  17.206315994262695,
  2.6987156867980957,
  4.148049831390381,
  1.3388659954071045],
 [54.27294158935547,
  54.27294158935547,
  1.1927053928375244,
  7.3670172691345215,
  0.3939455449581146],
 [17.102169036865234,
  17.102169036865234,
  1.5074857473373413,
  4.135477066040039,
  0.710621178150177],
 [0.22102712094783783,
  0.22102712094783783,
  0.2077571600675583,
  0.4701352119445801,
  0.09438832104206085],
 [0.21153846383094788,
  0.21153846383094788,
  0.17307692766189575,
  0.4599331021308899,
  0.08788689970970154],
 [0.21153846383094788,
  0.21153846383094788,
  0.17307692766189575,
  0.4599331021308899,
  0.08788689970970154],
 [912.955322265625,
  912.955322265625,
  17.154495239257812,
  30.215150833129883,
  5.4415483474731445],
 [0.3832080662250519,
  0.3832080662250519,
  0.31358927488327026,
  0.6190380454063416,
  0.1555105745792389],
 [1.538053274154663,
  1.538053274154663,
  0.407085120677948,
  1.2401827573776245,
  0.2162237912416458],
 [512.0506591796875,
  512.0506591796875,
  12.544473648071289,
  22.628536224365234,
  4.331240177154541],
 [548.33740234375,
  548.33740234375,
  14.756406784057617,
  23.41660499572754,
  5.457037925720215],
 [19.79263687133789,
  19.79263687133789,
  0.9327372312545776,
  4.448891639709473,
  0.36923515796661377],
 [3806.962646484375,
  3806.962646484375,
  40.09707260131836,
  61.70058822631836,
  10.067750930786133],
 [180.47222900390625,
  180.47219848632812,
  5.106348037719727,
  13.43399429321289,
  1.944958209991455],
 [35.82570266723633,
  35.82570266723633,
  1.4716256856918335,
  5.985457420349121,
  0.6321569681167603],
 [0.23169037699699402,
  0.23169037699699402,
  0.20639093220233917,
  0.48134225606918335,
  0.09886818379163742],
 [0.22428171336650848,
  0.22428171336650848,
  0.18873141705989838,
  0.4735839068889618,
  0.09470772743225098],
 [102.98871612548828,
  102.98871612548828,
  2.448349952697754,
  10.148335456848145,
  0.8332995176315308]]

In [2]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])
sum(test_mae)/10

In [3]:
test_mae

[2.059077262878418,
 0.17307692766189575,
 0.19646000862121582,
 7.241370677947998,
 0.17307692766189575,
 0.17307692766189575,
 0.2244052141904831,
 0.17307692766189575,
 0.17307692766189575,
 0.6093320250511169,
 0.1665557473897934,
 1.2514338493347168,
 2.6987156867980957,
 1.1927053928375244,
 1.5074857473373413,
 0.2077571600675583,
 0.17307692766189575,
 0.17307692766189575,
 17.154495239257812,
 0.31358927488327026,
 0.407085120677948,
 12.544473648071289,
 14.756406784057617,
 0.9327372312545776,
 40.09707260131836,
 5.106348037719727,
 1.4716256856918335,
 0.20639093220233917,
 0.18873141705989838,
 2.448349952697754]

In [26]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 215.24067432234685
avg mae: 3.8064714396993318
avg are: 1.3057692368825278


In [27]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 2.9121719002723694
median testing mse: 3.4831897020339966


In [28]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.4711673706769943
median testing mae: 0.5082085728645325


In [29]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.1538461595773697


In [30]:
test_ARE

[0.4903846,
 0.15384616,
 0.15384616,
 2.6057692,
 0.15384616,
 0.15384616,
 0.15384616,
 0.15384616,
 0.15384616,
 0.15384616,
 0.13461539,
 0.48076922,
 0.8173077,
 0.15384616,
 0.5769231,
 0.15384616,
 0.15384616,
 0.15384616,
 5.1153846,
 0.15384616,
 0.15384616,
 5.076923,
 4.9134617,
 0.26923078,
 13.307693,
 1.9038461,
 0.25,
 0.15384616,
 0.15384616,
 0.7692308]